# III: Bayesian HCI: A review and a reflection

## Outcomes

You will understand:

* The loops and stakeholders in interaction design: modelling, design, interaction and analysis
* Key themes of Bayesian approaches in HCI
* 1. How to apply Bayesian models at interaction time: **inference of intent**
* 2. Design time optimisation with uncertainty: **Bayesian optimisation**
* 3. How to analyse experimental evaluations: **Bayesian statistics for empirical analyses**
* 4. How to model cognitive process using Bayesian models: **Bayesian cognitive modelling**
* 5. How to apply interaction to problems of Bayesian modelling




## The times, the players and the loops
![The times, the players and the loops](imgs/bayesian_loops.png)

### An HCI process
What goes on in HCI? An old-school traditional HCI process might unfold like this:

* A **psychologist** builds models of how people perceive and act in the world, and how they understand and might operate interfaces.
* A **designer** thinks about a problem and uses tools like use cases, personas, scenarios to conceptualise how an interface might used; and prototyping tools (like paper prototypes) to decide how that interface should be used.
* An interface is implemented. When it is in operation, a user is engaged in a **control loop** where the interface reacts to inputs from sensors, changes internal states, and updates outputs on displays.
* To determine how well this interface works, an **evaluation** is conducted to analyse the way in which the interface is used, how successfully it is operated, how users feel about different aspects, etc.


### The times and the loops
This gives rise to four distinct **times**:

* **Modelling time** which models human behaviour in the presence of interactive systems;
* **Design time** where a specific instantiation of an interface is arrived at by some design process, informed by modelling;
* **Interaction time** where specific user goals at an instant of time are serviced via an interface;
* **Analysis time** where the efficacy of the interface is reviewed and analysed.

None of these are simple feed-forward processes; all would typically involve *feedback*; we might test prototypes to refine designs or to build better models of interface usage.

### Themes

When we talk through these, there are some key themes of "Bayesianism" to look for:

* Models: what form do the data generating processes take? What are the parameters that can vary?
* Inference: how is it achieved? What computations give us posteriors, and how is evidence brought into play?
* Uncertainty: how is represented and what is used for? Why is uncertainty valuable in this context?
* Priors: where do they come from and how are they specified?

### Commonalities
A Bayesian approach:
* *always* involves manipulating probability distributions over unobserved parameters;
* *always* involves a movement from prior to posterior guided by belief;
* *always* requires defining a data generating process;
* *always* preserves uncertainty (to some degree, usually limited by computation);
* *always* has some computational struggles!


### Structure

In each sub-section, I'll:
* outline the concepts and how Bayesian ideas apply
* briefly discuss 2-3 papers from CHI or UIST that used these ideas in real research applications.
* then we'll run through a self-contained live code example.

## In: In-the-loop: Bayesian interaction

### Summary
We infer distributions over intentions during an interaction, fusing together evidence from multiple sources, and over time. We need models of behaviour/sensing/display, representations of probability distributions that are compatible with our UI software, and techniques for preserving and reflecting probabilistic state.


### Motivation
Adding probabilistic Bayesian inference in the interaction loop can make interaction more robust, rational and efficient.

* **Robust** means that the interaction should reliably coincide with intention, even in the presence of disturbance;
* **Rational** means that the actions should be taken that accurately reflect both certainty and utility;
* **Efficient** means that action should coincide with intention with a minimum of time, mental or physical effort expended.

This is particularly salient when there is a large control:action disparity.
* For example: the interaction technique is marginally effective (e.g. in assistive technology for text entry) 
* For example: the space of possible actions is huge (e.g. a search engine)

### Model

A human interacting with a computer. We can see this several ways:

[IMAGE]

* Communication: users encode selections to be packaged over a motor channel and then a sensor channel. These are decoded into state changes.
    * Essentially feed-forward; low-latency, high mental cost, high-bandwidth. Domain of pattern recognition approaches.
* Control: users drive a system into a desired equilibrium by feedback control, through a motor/sensor channel, via a mediating mechanism (like a cursor), and back through a perceptual channel.
    * Essentially closed-loop. Low mental cost, low bandwidth. Domain of traditional UI components.
* Inference: A system infers what a user might want to happen, based on observed evidence, and designs experiments via the feedback channel to optimally acquire more.
    * Closed-loop or feed-forward. Domain of probabilistic interfaces.


##### Interaction as inference
If we view interaction as inference of intention, there are three elements:
* **Interaction is inference**; it is the process of inferring a distribution over a hidden variable: what the user wants a system to do. 
* **Observations are indirect, noisy and incomplete** What a system sees is a distorted and incomplete representation of user actions in the world, which are in turn a noisy representation of internal intentions (your hand does not always go where you want it...)
* **Interaction occurs over time** Interaction is a *process* that evolves over time. Information flow is not instantaneous. Observations must be fused together to update beliefs.

### Approach

* Inference approach: Model the system's uncertainty over user intentions as a probability distribution, update via Bayes' rule. $P(\text{intention}|{input})$
    * What inputs would we *expect* to see, given hypothesised intentions and their prior likelihood?
        * For example, what mouse trajectories would we expect to see, given that a user was pointing a specific icon?
    * If still unresolved, provoke a response that will maximise the information gain (change in entropy).
        * For example, move one of the two most likely icons and see if the user compensates.

We'll look at a **Bayesian** approach to modelling human computer interaction, where we explicitly model what might be going on inside a user's mind and use Bayesian methods to try and perform "optimal mindreading". 
<img src="imgs/brain_inference.png" width="70%">

### Models
This view on interaction sees user intentions as **unknown values** which are partially observed through inputs. The time series of inputs from the user give a partial, noisy, incomplete view of intention inside the user's head, along with a great deal of superfluous information. 

We try and infer intention *generative model* which is a simplified representation of intention and how it is mediated and transformed by the world. The stronger model we have available, the more effectively we can infer intention.

> In this view, improving interaction (or at least *input*) comes down to more efficiently concentrating probability density where a user wants it. A better pointing device reduces uncertainty faster; a better display helps a user understand how best to target future actions to concentrate belief as desired; a better model of the user intentions concentrates belief with less explicit effort on the part of a user.

<img src="imgs/contraction_probability.png" width="70%">

#### Partitioning the inferred variables

We can further partition the problem. The causes of observed evidence can be factored, for example, into:

* **Mind state** The parameters of the intentions that generate the behaviour: what menu option does the user want?
* **World state** The parameters of the motor system that generate movement: where is the user's hand?
* **Sensor state** The parameters of the sensing system that generates signals: what is the camera matrix?

$$P(X_{\text{intention}}, X_{\text{motor}}, X_{\text{sensing}}|Y)$$

[Betancourt's article on probabilistic modeling](https://betanalpha.github.io/assets/case_studies/modeling_and_inference.html) expresses these ideas in terms of the "phenomenon" (intention), "environment" (motor/world system) and "probe" (sensing/interface context).

##### Purity
It's rare to have an interaction loop that is *pure* Bayesian, where every step of the interaction is modelled with probability distributions and updated via Bayesian inference. Often we restrict the Bayesian slice to the realm where uncertainty is most relevant.

[DIAGRAM]

For example, we might have a system where:

* A standard ML algorithm processes high-dimensional sensor data to a simpler form (e.g. a touch sensor to a cursor)
* Bayesian inference over possible gestures that could be being performed by the cursor (not the sensor)
* A standard state machine which actuates when probabilities cross thresholds.

This gives the power of inverse models (e.g. deep nets) in efficiently performing *representation learning*, and the standard and familiar operation of systems with discrete states, but with some of the robustness and flexibility of a Bayesian model. It's not *necessary* to do this; we could have a fully Bayesian interactive system, but it's rarely practical to do so, and it raises questions about how a user might understand and use such a system.

### Forward-inverse
[FORWARD-INVERSE]
In particular, this is a really powerful way of plugging together advanced ML systems (e.g. for computer vision); these are extremely powerful, but typically don't have any representation of uncertainty, or at best a weak one. Instead, most deep learning based systems map directly from one space to another; in interaction this is often a mapping from an observed sensor state (like an image) to an inferred hidden state (like the pose of the person in the image). This is an **inverse model** (because it maps from observations to hidden states); we can also use ML-learned **forward models** (i.e. generative models from hidden states to models) as part of a Bayesian model, learning some elements of the data generating process that we cannot easily write down analytically.

### Uncertainty
* Input from a human user will often be ambiguous, at least at some point in time. This might be because of:
    * Genuine noise in the sensing that disturbs the intended control;
    * User confusion, error or changing intentions;
    * Partial and incomplete evidence of intention from sensing;
    * Inaccurate models of behaviour given intention.
* Failure to represent uncertainty means actions might be taken without sufficient evidence, or require unreasonable quantities of evidence to actuate.
* Balancing the information flow requires a proper accounting of uncertainty.

[DIAGRAM]

#### Feedback of uncertainty
Uncertainty is useful to perform robust inference. It can also assist users in understanding the belief state of the system they are controlling. Reflecting the probability distribution (or summaries of it) can be a powerful way of building interfaces that help a user understand how their action is being interpreted, and when ambiguity remains.

#### Active inference
A system which models uncertainty can take actions to minimise it; it's not possible to do this if you don't capture the uncertainty! This means we can build interactive systems that generate stimuli to optimally acquire information (reduce uncertainty). This is the "pull" to the "push" of uncertainty feedback. A Bayesian interactive system can reveal its probability distribution to help the user feed it the right information, and stimulate the user to feed it the most helpful information. 


### Priors
Typically we have priors that are either derived from:
    * observations from a population; for example, we might use historical frequencies of selection of menu items as a prior for the probability of selecting those items
    * the immediate past; we use a sequential filtering process where the posterior from one timestep becomes the prior for the next; for example, a language model might predict the next character to enter *given* the previous characters entered.

We may also have priors that arise from psychological or physiological models. For example, if we were tracking someone's hand in space, we could use a prior that implemented the reach volume of the hand -- the probability of the hand being more than 2m from the torso is probably small, for example.


## Example
We'll look at building a very simple gesture recogniser using a *particle filter* (that's a sample-based/MCMC probabilistic filter).

**Link to the notebook: [examples/1_in_the_loop.ipynb](ex_1_in_the_loop.ipynb)**

## Papers 

### Paper a: AnglePose
1. Rogers, Simon, John Williamson, Craig Stewart, and Roderick Murray-Smith. 2011. **“AnglePose: Robust, Precise Capacitive Touch Tracking via 3d Orientation Estimation.”** In *Proceedings of the 2011 Annual Conference on Human Factors in Computing Systems - CHI ’11*, 2575. Vancouver, BC, Canada: ACM Press. https://doi.org/10.1145/1978942.1979318.

![AnglePose](imgs/paper_1_anglepose.png)

* **Distribution**
    * Over finger poses, i.e. over the 4D vector space $[x,y, \phi, \theta]$. Roll is ignored.
* **Data Generating Process**
    * Simple model of finger as a hinged flap
    * Observed via basic capacitive model 
    * First-order dynamics of motion (e.g. velocity tends to be constant in the short term)
* **Priors**
    * The previous time step (i.e. the last pose known)
    * And the bounds of the device
* **Observation**
    * Raw sensor images (160 element vectors representing capacitances)
* **Inference**
    * Sequential Monte Carlo (particle filter) -- propagating samples forward in time.
* **Interaction benefit**
    * Increased precision and robustness of pointing
    * Uncertainty available to decide on whether to actuate

### Paper b: Dasher
1. Ward, David J., Alan F. Blackwell, and David JC MacKay. 2000a. **“Dasher-a Data Entry Interface Using Continuous Gestures and Language Models.”** In *UIST*, 129–37.

![Dasher](imgs/paper_2_dasher.png)

* **Distribution**
    * Over (sequences of) characters of an alphabet.
* **Data Generating Process**
    * Characters are generated by a Markov model (PPM-based) conditioned on prior characters.
    * These are indexed based on a response to a stimuli which dedicates screen real-estate according to prior.
* **Priors**
    * Language model given previous characters    
* **Observation**
    * Cursor position (note: not stochastic!).
* **Inference**
    * Exact.
* **Interaction benefit**
    * Increased efficiency of interaction 
    * Easy to fuse with other input sources (e.g. SpeechDasher)

### Paper c: BIGNav
1. Liu, Wanyu, Rafael Lucas d’Oliveira, Michel Beaudouin-Lafon, and Olivier Rioul. 2017a. **“Bignav: Bayesian Information Gain for Guiding Multiscale Navigation.”** In *Proceedings of the 2017 CHI Conference on Human Factors in Computing Systems*, 5869–80. ACM.

[FILMSTRIP]

* **Distribution**
    * Over characters of an alphabet.
* **Data Generating Process**
    * Characters are generated by a Markov model (PPM-based) conditioned on prior characters.
    * These are indexed based on a response to a stimuli which dedicates screen real-estate according to prior.
* **Priors**
    * Language model given previous characters    
* **Observation**
    * Cursor position (note: not stochastic!).
* **Inference**
    * Exact.
* **Interaction benefit**
    * Increased efficiency of interaction 
    * Easy to fuse with other input sources (e.g. SpeechDasher)


## Before: Bayesian optimisation at design time


### Motivation
Design by hand-engineering relies on idiom and experience, which may not be optimal or agile enough to adapt to new contexts, such as new devices or specific user groups. Automatic optimisation of the specific parameterisation of an interface can address this *but* acquiring data from users to fine-tune is very expensive and very noisy. Bayesian optimisation is a sample-efficient way to tune designs by constructing a distribution over proxy objective functions.

#### Approach


* Uncertainty: 


Motivation
Diagram
### Example
### Papers
#### Paper a: Engaging games
Khajah, Mohammad M, Brett D Roads, Robert V Lindsey, Yun-En Liu, and Michael C Mozer. 2016. **Designing Engaging Games Using Bayesian Optimization.** In *Proceedings of the 2016 CHI Conference on Human Factors in Computing Systems*, 5571–82.

#### Paper b: Better fonts
Kadner, Florian, Yannik Keller, and Constantin Rothkopf. 2021. **Adaptifont: Increasing Individuals’ Reading Speed with a Generative Font Model and Bayesian Optimization.** In *Proceedings of the 2021 CHI Conference on Human Factors in Computing Systems*, 1–11.

#### Paper c: Crowdsourced design
Dudley, John J, Jason T Jacques, and Per Ola Kristensson. 2019. **Crowdsourcing Interface Feature Design with Bayesian Optimization.** In *Proceedings of the 2019 Chi Conference on Human Factors in Computing Systems*, 1–12.

### Example

We'll look at optimising a... using a Gaussian process as a proxy for our objective function with exact inference. This is a simple to apply and widely applicable way of modelling preference or performance functions.

**Link to the notebook: [examples/bayesian_optimisation.ipynb](ex_2_bayesian_optimisation.ipynb)**


## After: Bayesian analysis of empirical work
### Introduction
When we evaluate how an interface works, we typically rely on statistical tools to analyse quantitative results. Even better, we'd hope to use statistical tools to *design* the analyses
that we intend to perform: statistics starts before the experiment, not afterwards!

Statistical analysis in HCI are common -- just look at any CHI paper -- but they are often inappropriate or flawed. Almost all HCI uses classical frequentists statistics adopted from psychology. Frequentist statistics include things like t-tests, ANOVAs, Mann-Whitney tests, and so on. Frequentist statistics *are not wrong*; they are perfectly mathematically valid and sometimes useful techniques. But they are most certainly not the only way to do statistics and rarely the appropriate methods for HCI.

> Some authors (e.g. E. T. Jaynes or Aubrey Clayton) would argue that all of frequentists statistics is useless rubbish and that Bayesian methods are the only valid approach to analysing data. 
> They might be right. 


### Motivation
Why do frequentist statistics fail to do what we want to do? How do Bayesian methods help? And what are the trade-offs?

> Do we want to compare two blocks of observations, apply a "black box" algorithm, and make statements about how likely the variation is to be "random" as opposed to "true"?

#### False dichotomy and backwards questions
* Frequentist statistics often force us into dichotomous decisions: is A (statistically significantly) better than B. 
* But *constructing* a B to compare with may not make much sense. At the very least, it leaves make pointwise comparisons to baselines.
* This mode of thought permeates how we think about experimental work in HCI **but it is not necessary**.
* Dichotomous analyses *are* appropriate if you want to make comparisons to a baseline, in controlled studies, where effects are large and false positives are important.
    * For example, in randomised controlled trials for efficacy of drugs.
* Frequentist statistics can't answer many questions we'd like to know the answer to; "how likely is it that A is twice as efficient as B"? 
    * **Our analysis procedures should answer the questions we want to know, not define the questions we are allowed to ask!**
* Frequentist statistics typically give us a single estimate but not uncertainty about inference.
    * FS can make only make uncertainty quantifications about observations, not parameters.
    * But our questions are usually about parameters!

#### Application and interpretation
* Frequentist statistics produce quantities like p-values and confidence intervals. **These are valid but very hard to interpret!**
    * Q: define one of these terms *correctly*    

* Frequentist statistics are easy to misuse; great care is needed make sure you account for multiple testing, researcher degrees of freedom, stopping rules to preserve the false positive rate of testing procedures.
* Frequentist statistics come in pre-packaged forms (e.g. ANOVA). These have to be slotted in. This leads to several problems:
    * You must make the choice of approach intelligently among a zoo of tests and procedures. But you cannot adjust any of the details to fit your problem.
    * No modelling is required and so you can avoid defining a good data generating process -- which means you are likely to do silly things.
* No prior information is explicitly defined. That seems "objective", but it hides that every frequentist approach makes *hidden* assumptions equivalent to priors.
    * In other words, you get a one-size-fits-all prior that is unlikely to match your true priors, and there is nothing you can do about it.


#### Other issues
* Frequentist methods are typically used because of tradition. This is not a good reason.
    * Bayesian statistics are not widely understood.
    * Researchers are reluctant to admit that much of published statistics might be dubious.
    * Sotfware and tooling doesn't make it easy to Bayesian analysis.
    * Philosophical debates still rage about Bayesian vs. frequentist models. Some are still cautious about Bayesian methods (or aligned with the dark side).
* Computational issues used to make Bayesian methods impractical. This isn't a good excuse these days.
* Priors can be controversial. You can obviously change what you assume to change what you believe in the light of evidence.
    * But the counter-argument is that *expliclity stating* your assumptions (as priors) is the right way to be honest; a procedure can't guarantee honesty but it can hide deception.






### Papers
#### Paper a: Why Bayesian Statistics Better Fit the Culture and Incentives of HCI
Kay, Matthew, Gregory L Nelson, and Eric B Hekler. 2016.
**Researcher-Centered Design of Statistics: Why Bayesian Statistics
Better Fit the Culture and Incentives of HCI.** In *Proceedings of the
2016 CHI Conference on Human Factors in Computing Systems*, 4521–32.

##### The problem identified

##### The solution suggested

##### Why Bayesian?



#### Paper b: Prior selection
Phelan, Chanda, Jessica Hullman, Matthew Kay, and Paul Resnick. 2019. **Some Prior (s) Experience Necessary: Templates for Getting Started with Bayesian Analysis.** In *Proceedings of the 2019 CHI Conference on Human Factors in Computing Systems*, 1–12.

##### The problem identified

##### The solution suggested

##### Why Bayesian?


#### Paper c: Dichotomous-ness
Besançon, Lonni, and Pierre Dragicevic. 2019. **The Continued Prevalence of Dichotomous Inferences at CHI.** In *Extended Abstracts of the 2019 CHI Conference on Human Factors in Computing Systems*, 1–11.

##### The problem identified

##### The solution suggested

##### Why Bayesian?

### Example

### Example

We'll look at analysing a ..., from a pre-created dataset. We'll use a simple regression model, using `pymc3` to perform MCMC-based inference. We'll look at how we can interpret and report the results of the inference process.

**Link to the notebook: [ex_3_regression_analysis.ipynb](ex_3_regression_analysis.ipynb)**



## About: Bayesian models of cognition and behaviour
### Introduction
How do we start to build interfaces? In the old days, hardware and computational power dictated many of the constraints.
![Ye olde interaction experience.]()
Even so, ancient mainframes still  had to group their switches logically, label them, and lay them out for ergonomic reachability. 
To build interfaces we need to understand how humans behave. Human behaviour is the key constraint that defines interaction. We need models of how humans behave. 

Some HCI relies on human experience of behaviour ("designer's intuition"), heuristics that formalise these experiences, and repeated evaluation as a crutch to stitch up the mismatches. Computational interaction rejects this approach, and puts computational models first. This means we have to have actionable simulators of aspects of human behaviour that can be applied to interface design problems.

Some human simulation problems are tricky but largely in the realm of the observable and relatively certain, like simulating reach volumes from biomechanics. But others, especially those that involve cognition, are very hard to construct. Cognitive processes are:

* weakly observable -- we have essentially no instruments to act upon them and limited modes of observation.
* potentially highly variable among the population and even within an individual
* Cognition is stochastic in production of evidence ("same" mental state => different observed outcomes).
* very complex in nature. A linear system probably won't suffice!

Diagram

### Papers
#### Paper a: Models of visualisation
Kim, Yea-Seul, Logan A Walls, Peter Krafft, and Jessica Hullman. 2019. **“A Bayesian Cognition Approach to Improve Data Visualization.”** In *Proceedings of the 2019 Chi Conference on Human Factors in Computing Systems*, 1–14.

#### Paper b: ABC for cognitive models

Kangasrääsiö, Antti, Jussi PP Jokinen, Antti Oulasvirta, Andrew Howes, and Samuel Kaski. 2019. **“Parameter Inference for Computational Cognitive Models with Approximate Bayesian Computation.”** *Cognitive Science* 43 (6): e12738.


### Example
No example for this section; it's just too much to cover in a short example -- but see Andrew's talk on Wednesday instead.


## With: Interaction with Bayesian models
### Introduction
Let's turn the tables. What can HCI offer Bayesian modelling? How could HCI help:

* create Bayesian models?
* validate and verify Bayesian models?
* visualise, explore and explain their implications and consequences?

> Bayesian modelling is often seen as hard, and the models as complex and inscrutable. This isn't "really" true, but
> there lots to be done to make the "real" complexity manageable. These are problems of interaction between users and models.

#### End users


Motivation
Diagram

### Papers
#### Paper a: Animation to reveal uncertainty
1. Kale, Alex, Francis Nguyen, Matthew Kay, and Jessica Hullman. 2018. **“Hypothetical Outcome Plots Help Untrained Observers Judge Trends in Ambiguous Data.”** *IEEE Transactions on Visualization and Computer Graphics* 25 (1): 892–902.
#### Paper b: A Bayesian workflow
1. Gelman, Andrew, Aki Vehtari, Daniel Simpson, Charles C. Margossian, Bob Carpenter, Yuling Yao, Lauren Kennedy, Jonah Gabry, Paul-Christian Bürkner, and Martin Modrák. 2020. **“Bayesian Workflow.”** *arXiv Preprint arXiv:2011.01808*. https://arxiv.org/abs/2011.01808.

### Example

**Link to the notebook: [examples/visualisation.ipynb](ex_5_bayesian_visualisation.ipynb)**